In [1]:
import gc
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import seaborn as sns
import subprocess
import tensorflow as tf
import torch
import torchvision.transforms as T

from collections import Counter
from datasets import ClassLabel, Dataset, Features, Image as DatasetsImage, concatenate_datasets, load_dataset
from datetime import datetime
from functools import partial
from io import BytesIO
from pathlib import Path
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, log_loss
from torch import nn
from torch.nn import functional as F
from torch.optim import LBFGS
from torchvision.transforms import ToPILImage
from tqdm import tqdm
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer
)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-05-03 16:33:02.288342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746289982.310914    4593 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746289982.318394    4593 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746289982.337410    4593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:174628998

In [3]:
# # --------------------------
# # GPU Environment Setup for Multi-GPU Optimization (GPUs 0-n)
# # --------------------------
# # Limit process to specific GPUs
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" #0, 1, 2, 3,...n
# print("Process restricted to GPUs:", os.environ["CUDA_VISIBLE_DEVICES"])

# # Ensure pip executables are available
# os.environ["PATH"] = f"{os.path.expanduser('~/.local/bin')}:" + os.environ["PATH"]

# # Enable memory growth for TensorFlow
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled on GPUs.")
#     except RuntimeError as e:
#         print("Error configuring GPUs:", e)
# print("GPUs available to this process (as seen by TensorFlow):", tf.config.list_physical_devices('GPU'))

# # Optional: Monitor current GPU usage
# gpu_usage = subprocess.check_output(["nvidia-smi"]).decode("utf-8")
# print("Current GPU usage:\n", gpu_usage)

In [14]:
# --------------------------
# 0. Global Configurations
# --------------------------
RUN_INFERENCE = True  # Toggle this off to disable running inference
IMAGE_DIR = "/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset"
BASE_PATH = IMAGE_DIR

LABEL_NAMES = [
    "anger", "disgust", "fear", "happiness",
    "sadness", "surprise", "neutral", "questioning"
]
id2label = dict(enumerate(LABEL_NAMES))
label2id = {v: k for k, v in id2label.items()}

VALID_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

def is_valid_image(filename):
    return filename.lower().endswith(VALID_EXTENSIONS) and not filename.startswith("._")

label_mapping = {name.lower(): name for name in LABEL_NAMES}

# 🔢 Dynamically determine the next version
def get_next_version(base_dir):
    existing = [
        os.path.basename(d) for d in glob.glob(os.path.join(base_dir, "V*_vit_final_independent"))
    ]
    versions = [int(d[1:].split("_")[0]) for d in existing if d.startswith("V") and "_" in d]
    next_version = max(versions, default=0) + 1
    return f"V{next_version}"

# Automatically create a versioned output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION = get_next_version("/home/ubuntu/MLexpressionsStorage")
VERSION_TAG = VERSION + "_" + timestamp
SAVE_DIR = os.path.join("/home/ubuntu/MLexpressionsStorage", VERSION_TAG)
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Output directory created: {SAVE_DIR}")

📁 Output directory created: /home/ubuntu/MLexpressionsStorage/V8_20250503_190001


In [16]:
# --------------------------
# 1. Auto-Load Latest Pretrained Model and Processor
# --------------------------

# Automatically load latest model path
MODEL_ROOT = "/home/ubuntu/MLexpressionsStorage"
model_dirs = sorted(
    [os.path.join(MODEL_ROOT, d) for d in os.listdir(MODEL_ROOT) if d.startswith("V") and os.path.isdir(os.path.join(MODEL_ROOT, d))],
    key=os.path.getmtime,
    reverse=True
)

if not model_dirs:
    raise FileNotFoundError("❌ No model folders found matching pattern 'V*' in MODEL_ROOT.")
model_path = model_dirs[0]
print(f"✅ Auto-loaded model from: {model_path}")

# Load base model and processor
model = AutoModelForImageClassification.from_pretrained(model_path)
processor = AutoImageProcessor.from_pretrained(model_path)

# Replace classification head to match current label schema
model.classifier = torch.nn.Linear(model.classifier.in_features, len(id2label))
model.config.id2label = id2label
model.config.label2id = label2id
model.config.num_labels = len(LABEL_NAMES)

# Define device and push model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

✅ Auto-loaded model from: /home/ubuntu/MLexpressionsStorage/V8_20250503_190001


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [13]:
# # --------------------------
# # Rescue & Save from Last Checkpoint
# # --------------------------

# # in case model save fails, resume from latest checkpoint
# processor.save_pretrained(SAVE_DIR)
# print("✅ Processor manually re-saved.")

# # ✅ Set correct version path (edit when needed)
# SAVE_DIR = "/home/ubuntu/MLexpressionsStorage/V8_20250427_155958"  # replace with correct version

# # 🔍 Locate latest checkpoint inside SAVE_DIR
# checkpoint_dirs = [
#     os.path.join(SAVE_DIR, d) for d in os.listdir(SAVE_DIR)
#     if d.startswith("checkpoint-") and os.path.isdir(os.path.join(SAVE_DIR, d))
# ]
# if not checkpoint_dirs:
#     raise ValueError("❌ No checkpoint found to recover from.")

# # to set checkpoint manually:    
# # latest_checkpiont = os.path.join(SAVE_DIR, "checkpoint-22500")  # or checkpoint-18000

# latest_checkpoint = max(checkpoint_dirs, key=os.path.getmtime)
# print(f"✅ Found latest checkpoint: {latest_checkpoint}")

# # 🧠 Load model & processor from checkpoint
# model = AutoModelForImageClassification.from_pretrained(latest_checkpoint)
# processor = AutoImageProcessor.from_pretrained(SAVE_DIR)

# # 🧯 Move model to CPU (to avoid GPU I/O hangs)
# model = model.to("cpu")

# # 💾 Save model state_dict
# torch.save(model.state_dict(), os.path.join(SAVE_DIR, "final_model.pth"))
# print(f"✅ State dict saved to: {SAVE_DIR}/final_model.pth")

# # 💾 Save full model and processor in Hugging Face-style
# model.save_pretrained(SAVE_DIR, safe_serialization=True)
# processor.save_pretrained(SAVE_DIR)
# print(f"✅ Model and processor saved to: {SAVE_DIR}")

# # 💾 (Optional) Save Trainer backup
# try:
#     trainer.save_model(os.path.join(SAVE_DIR, "backup_trainer_model"))
#     print(f"✅ Trainer backup saved to: {os.path.join(SAVE_DIR, 'backup_trainer_model')}")
# except Exception as e:
#     print(f"⚠️ Failed to save trainer backup: {e}")

# # 🧹 Clean up memory
# del model
# gc.collect()
# torch.cuda.empty_cache()
# print("✅ Memory cleanup complete.")

In [6]:
# --------------------------
# 2. Load and Prepare Dataset
# --------------------------
features = Features({
    'image': DatasetsImage(),
    'label': ClassLabel(names=LABEL_NAMES)
})

dataset = load_dataset(
    "imagefolder",
    data_dir="/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset",
    split="train",
    cache_dir="/tmp/hf_cache",
    features=features  # ✅ manually define all 8 labels
)

counter = {"n": 0}

def reconcile_labels(example):
    counter["n"] += 1
    if counter["n"] % 1000 == 0:
        print(f"Processed {counter['n']} images...")
        
    # If the label is already an integer, convert it to a string using the dataset features.
    if isinstance(example["label"], int):
        # Use dataset.features["label"].int2str to get the string label.
        original_label = dataset.features["label"].int2str(example["label"]).strip().lower()
    else:
        original_label = example["label"].strip().lower()

    # Map the lowercased label to the pre-trained model's expected label.
    pretrain_label = label_mapping.get(original_label)
    # If not recognized, mark it for filtering.
    # Covert the mapped label to its corresponding integer.
    example["label"] = label2id[pretrain_label] if pretrain_label is not None else -1
    return example

# Apply reconciliation function to dataset.
dataset = dataset.map(reconcile_labels, desc="Re-labeling dataset", with_indices=False, num_proc=8)
# Filter out any examples that were marked as unrecognized.
dataset = dataset.filter(lambda x: x["label"] != -1)
print("Total examples after filtering:", len(dataset))

Total examples after filtering: 32041


In [11]:
# --------------------------
# 2A. Dataset Label Overview and Folder Stats
# --------------------------
def analyze_dataset_structure(dataset, id2label, base_path):
    # Print label schema from the dataset
    print("Label schema (from dataset):", dataset.features["label"])

    # Label distribution from the dataset object
    label_counts = Counter(dataset["label"])
    print("\n📊 Full dataset label distribution (from Dataset object):")
    for label_id, count in sorted(label_counts.items()):
        print(f"  {id2label[label_id]}: {count} examples")

    # Dynamically detect minority classes (lowest 3 frequencies)
    N = 3
    minority_classes = set(
        label for label, _ in sorted(label_counts.items(), key=lambda x: x[1])[:N]
    )
    print(f"\n⚠️  Dynamically identified minority classes: {[id2label[i] for i in minority_classes]}")

    # Count images per directory, and store for later validation
    folder_image_counts = {}
    print("\n📂 Image count per label folder:")
    for label in sorted(os.listdir(base_path)):
        label_path = os.path.join(base_path, label)
        if os.path.isdir(label_path):
            valid_images = [img for img in os.listdir(label_path) if is_valid_image(img)]
            folder_image_counts[label] = len(valid_images)
            print(f"  {label}: {len(valid_images)} images")

    return minority_classes, folder_image_counts

# Example usage right after dataset loading
minority_classes, folder_image_counts = analyze_dataset_structure(dataset, id2label, BASE_PATH)

Label schema (from dataset): ClassLabel(names=['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral', 'questioning'], id=None)

📊 Full dataset label distribution (from Dataset object):
  anger: 4702 examples
  disgust: 607 examples
  fear: 4671 examples
  happiness: 8869 examples
  sadness: 1221 examples
  surprise: 4491 examples
  neutral: 5996 examples
  questioning: 1484 examples

⚠️  Dynamically identified minority classes: ['disgust', 'sadness', 'questioning']

📂 Image count per label folder:
  .ipynb_checkpoints: 0 images
  anger: 4702 images
  disgust: 607 images
  fear: 4671 images
  happiness: 8869 images
  neutral: 5996 images
  questioning: 1484 images
  sadness: 1221 images
  surprise: 4491 images


In [8]:
# --------------------------
# 3. Define Data Augmentation and Preprocessing Transformation
# --------------------------

# Baseline augmentation
data_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.1, contrast=0.1)
])

# Stronger augmentation for minority classes
minority_aug = T.Compose([
    T.RandomResizedCrop(224, scale=(0.6, 1.0)),
    T.RandomHorizontalFlip(p=0.8),
    T.RandomRotation(30),
    T.ColorJitter(0.4, 0.4, 0.4, 0.2),
    T.RandomGrayscale(p=0.3),
    T.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
])

#factory function that returns another function -> tranform_function
def make_transform_function(processor, minority_classes):
    def transform_function(example):
        label = example["label"]
        aug_pipeline = minority_aug if label in minority_classes else data_augment

        if example["image"].mode != "RGB":
            example["image"] = example["image"].convert("RGB")

        augmented_image = aug_pipeline(example["image"])
        inputs = processor(augmented_image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = example["label"]
        return inputs
    return transform_function

#returned transform_function is applied to each dataset example inside .map()
#each time the dataset runs transform_function(example), it receives:
#inputs = {
#     'pixel_values': tensor,
#     'labels': label_int
# }
dataset = dataset.map(make_transform_function(processor, minority_classes))

In [9]:
# --------------------------
# 4. Train-Validation Split
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]
# # in case above code doesn't run:
# split_dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="label")
# train_dataset = split_dataset["train"]
# eval_dataset = split_dataset["test"]

In [10]:
# --------------------------
# 5. Balance Dataset
# --------------------------
label_target = 4500
balanced_subsets = []

# Dynamically calculate label counts
label_counts = Counter(train_dataset["label"])
print("Original label distribution:", label_counts)

for label, count in label_counts.items():
    subset = train_dataset.filter(lambda x: x['label'] == label, num_proc=8)
    if count > label_target:
        subset = subset.select(random.sample(range(len(subset)), label_target))
    elif count < label_target:
        multiplier = label_target // len(subset)
        remainder = label_target % len(subset)
        subset = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
    balanced_subsets.append(subset)

train_dataset = concatenate_datasets(balanced_subsets).shuffle(seed=42)
print("After balancing:", Counter(train_dataset['label']))

Original label distribution: Counter({3: 7110, 6: 4850, 0: 3761, 2: 3680, 5: 3594, 7: 1171, 4: 985, 1: 481})


Filter (num_proc=8): 100%|██████████| 25632/25632 [05:15<00:00, 81.32 examples/s] 


After balancing: Counter({1: 4500, 6: 4500, 3: 4500, 5: 4500, 7: 4500, 0: 4500, 4: 4500, 2: 4500})


In [13]:
# # --------------------------
# # 6 Oversample Underrepresented Classes
# # --------------------------
# def oversample_dataset(dataset):
#     label_counts = Counter(dataset['label'])
#     max_count = max(label_counts.values())
#     label_datasets = []

#     for label in sorted(label_counts):
#         subset = dataset.filter(lambda x: x['label'] == label, num_proc=8)
#         multiplier = max_count // len(subset)
#         remainder = max_count % len(subset)
#         oversampled = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
#         label_datasets.append(oversampled)

#     return concatenate_datasets(label_datasets).shuffle(seed=42)

# train_dataset = oversample_dataset(train_dataset)
# print("After oversampling:", Counter(train_dataset['label']))

In [15]:
# --------------------------
# 7. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir=SAVE_DIR,                   # Directory to save checkpoints and the final model
    eval_strategy="epoch",                 # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    save_total_limit=2,                    # ✅ (optional) Keep only last 2 checkpoints to save space
    learning_rate=4e-5,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir=os.path.join(SAVE_DIR, "logs"),  # ✅ Save logs inside versioned folder
    logging_strategy="epoch",                 # ✅ Log once per epoch
    save_safetensors=True                  # ✅ Optional: saves model weights in `.safetensors` (safe format)
)

In [16]:
# --------------------------
# 8. Define Compute and Confusion Metrics
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Define a compute_metrics function w/ confusion matrix logging
def compute_metrics_with_confusion(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    print("\nClassification Report:")
    print(classification_report(labels, preds, target_names=LABEL_NAMES))

    # Save raw values for further use if needed
    np.save(os.path.join(SAVE_DIR, f"logits_eval_{VERSION}.npy"), logits)
    np.save(os.path.join(SAVE_DIR, f"labels_eval_{VERSION}.npy"), labels)    

    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=LABEL_NAMES,
        yticklabels=LABEL_NAMES
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"confusion_matrix_epoch_{VERSION}.png"))
    plt.close()

    # Identify top 3 confused class pairs (excluding diagonal)
    confusion_pairs = []
    for i in range(len(LABEL_NAMES)):
        for j in range(len(LABEL_NAMES)):
            if i != j:
                confusion_pairs.append(((LABEL_NAMES[i], LABEL_NAMES[j]), cm[i][j]))
    top_confusions = sorted(confusion_pairs, key=lambda x: x[1], reverse=True)[:3]
    print("\nTop 3 confused class pairs:")
    for (true_label, pred_label), count in top_confusions:
        print(f"  - {true_label} → {pred_label}: {count} instances")

    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

In [17]:
# --------------------------
# 9. Trainer with Class-Weighted Loss
# --------------------------

# Compute class weights from training set
label_freqs = Counter(train_dataset['label'])
total = sum(label_freqs.values())
class_weights = torch.tensor([total / label_freqs[i] for i in range(len(label_freqs))], dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Trainer to inject class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels, weight=class_weights)
        return (loss, outputs) if return_outputs else loss

# trainer initialization
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_with_confusion,
)

# Fine-tune model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.258900,0.330817,0.917304
2,0.075200,0.375850,0.925417
3,0.034400,0.458280,0.921829
4,0.010500,0.419278,0.934935
5,0.002100,0.424627,0.939148



Classification Report:
              precision    recall  f1-score   support

       anger       0.85      0.93      0.89       941
     disgust       0.80      0.85      0.82       126
        fear       0.90      0.86      0.88       991
   happiness       0.97      0.94      0.96      1759
     sadness       0.94      0.86      0.90       236
    surprise       0.93      0.94      0.94       897
     neutral       0.90      0.92      0.91      1146
 questioning       0.95      0.95      0.95       313

    accuracy                           0.92      6409
   macro avg       0.91      0.91      0.90      6409
weighted avg       0.92      0.92      0.92      6409


Classification Report:
              precision    recall  f1-score   support

       anger       0.91      0.90      0.91       941
     disgust       0.88      0.78      0.82       126
        fear       0.93      0.87      0.90       991
   happiness       0.97      0.94      0.96      1759
     sadness       0.93      0

TrainOutput(global_step=22500, training_loss=0.07624199735853407, metrics={'train_runtime': 25472.2209, 'train_samples_per_second': 7.067, 'train_steps_per_second': 0.883, 'total_flos': 1.394930822971392e+19, 'train_loss': 0.07624199735853407, 'epoch': 5.0})

In [ ]:
# # --------------------------
# # 10. Save Final Independent Model  - STUCK!
# # --------------------------
# model = model.to("cpu")   #move to CPU first to avoid GPU I/O freeze

# torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'final_model.pth'))
# model.save_pretrained(SAVE_DIR)
# processor.save_pretrained(SAVE_DIR)
# print(f"✅ Model and processor saved to {SAVE_DIR}")

# # Save full Trainer backup (optimizer state, scheduler, RNGs etc.)
# trainer.save_model(os.path.join(SAVE_DIR, "backup_trainer_model"))
# print(f"✅ Trainer backup saved to {os.path.join(SAVE_DIR, 'backup_trainer_model')}")

# # --------------------------
# # 10B. Fast Recovery Save (Optional Safety Net)
# # --------------------------
# from transformers import AutoModelForImageClassification

# # Locate the last checkpoint inside SAVE_DIR
# latest_checkpoint = max(
#     (os.path.join(SAVE_DIR, d) for d in os.listdir(SAVE_DIR) if d.startswith("checkpoint-")),
#     key=os.path.getmtime,
#     default=None
# )

# if latest_checkpoint:
#     print(f"✅ Found latest checkpoint at: {latest_checkpoint}")
#     recovered_model = AutoModelForImageClassification.from_pretrained(latest_checkpoint)
#     recovered_model.save_pretrained(SAVE_DIR)  # Save cleanly
#     processor.save_pretrained(SAVE_DIR)        # Save processor again just to be safe
#     print(f"✅ Re-saved model and processor from checkpoint to: {SAVE_DIR}")
# else:
#     print("⚠️ No checkpoint found for recovery.")

In [1]:
# --------------------------
# 10. Save Final Independent Model (Safe Save Mode)
# --------------------------

model = model.to("cpu")  # move to CPU first

# Save processor
processor.save_pretrained(SAVE_DIR)
print(f"✅ Processor saved to: {SAVE_DIR}")

# Save state dict
final_model_path = os.path.join(SAVE_DIR, 'final_model.pth')
torch.save(model.state_dict(), final_model_path)
print(f"✅ State dict saved to: {final_model_path}")

# Save full model
model.save_pretrained(SAVE_DIR, safe_serialization=True)
print(f"✅ Full model saved to: {SAVE_DIR}")

# Save trainer state (if defined)
if 'trainer' in globals():
    trainer.save_model(os.path.join(SAVE_DIR, "backup_trainer_model"))
    print("✅ Trainer backup saved.")
    
# Free memory
del model
gc.collect()
torch.cuda.empty_cache()
print("✅ Memory cleanup complete after save.")

NameError: name 'model' is not defined

In [17]:
# --------------------------
# 11. Inference Utilities
# --------------------------

# Single image prediction (unbatched)
def predict_label(image_path, threshold=0.85):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)
        conf, pred_idx = torch.max(probs, dim=-1)
    return (id2label[pred_idx.item()], conf.item()) if conf.item() >= threshold else ("REVIEW", conf.item())

# Batched prediction
def batch_predict(image_folder, batch_size=64, threshold=0.85):
    all_preds = []
    error_count = 0
    image_paths = [
        p for p in Path(image_folder).rglob("*")
        if is_valid_image(p.name)
    ]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Running inference in batches"):
        batch_paths = image_paths[i:i + batch_size]
        images, valid_paths = [], []

        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
                valid_paths.append(str(path))
            except Exception:
                error_count += 1
                continue

        if not images:
            continue

        inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)
            confs, preds = torch.max(probs, dim=-1)

        for pred, conf, path in zip(preds.tolist(), confs.tolist(), valid_paths):
            all_preds.append(LABEL_NAMES[pred] if conf >= threshold else "REVIEW")

    print(f"✅ Inference complete. Skipped {error_count} invalid image(s).")
    return all_preds

# Distribution plot
def plot_distribution(predictions, output_path):
    label_counts = Counter(predictions)
    labels = sorted(label_counts.keys())
    counts = [label_counts[label] for label in labels]

    plt.figure(figsize=(10, 5))
    plt.bar(labels, counts)
    plt.title("Predicted Expression Distribution")
    plt.xlabel("Expression")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

In [18]:
# --------------------------
# 12. Entry Point for Inference
# --------------------------
if __name__ == "__main__" and RUN_INFERENCE:

    # Auto-locate latest model directory
    OUTPUT_PATH = os.path.join(SAVE_DIR, f"{VERSION}_distribution_plot_{timestamp}.png")

    predictions = batch_predict(IMAGE_DIR)
    plot_distribution(predictions, OUTPUT_PATH)
    print(f"Distribution plot saved to: {OUTPUT_PATH}")

Running inference in batches: 100%|██████████| 501/501 [04:04<00:00,  2.05it/s]

✅ Inference complete. Skipped 0 invalid image(s).
Distribution plot saved to: /home/ubuntu/MLexpressionsStorage/V8_20250503_190001/V8_distribution_plot_20250503_190001.png


In [19]:
# --------------------------
# 13. Temperature Scaling Calibration 
# --------------------------

# Wrapper model for calibrated inference
class ModelWithTemperature(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, input_ids=None, pixel_values=None, **kwargs):
        logits = self.model(pixel_values=pixel_values).logits
        return logits / self.temperature

    def set_temperature(self, logits, labels):
        nll_criterion = nn.CrossEntropyLoss()
        optimizer = LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval_fn():
            optimizer.zero_grad()
            loss = nll_criterion(logits / self.temperature, labels)
            loss.backward()
            return loss

        optimizer.step(eval_fn)
        print(f"Optimal temperature (wrapped): {self.temperature.item():.4f}")
        return self

# Simple calibration-only function for analysis and storage
def apply_temperature_scaling(logits_path=f"{VERSION}_logits_eval.npy", labels_path=f"{VERSION}_labels_eval.npy"):
    if not (os.path.exists(logits_path) and os.path.exists(labels_path)):
        print("❌ Logits or labels not found. Skipping temperature scaling.")
        return None

    logits = torch.tensor(np.load(logits_path), dtype=torch.float32)
    labels = torch.tensor(np.load(labels_path), dtype=torch.long)

    class TemperatureScaler(nn.Module):
        def __init__(self):
            super().__init__()
            self.temperature = nn.Parameter(torch.ones(1) * 1.5)

        def forward(self, logits):
            return logits / self.temperature

    model = TemperatureScaler()
    optimizer = LBFGS([model.temperature], lr=0.01, max_iter=50)

    def eval_fn():
        optimizer.zero_grad()
        loss = F.cross_entropy(model(logits), labels)
        loss.backward()
        return loss

    optimizer.step(eval_fn)
    calibrated_logits = model(logits)
    probs = F.softmax(calibrated_logits, dim=1).detach().numpy()
    logloss = log_loss(labels.numpy(), probs)

    # Save optimal temperature
    temperature_value = model.temperature.item()
    torch.save(torch.tensor([temperature_value]), os.path.join(SAVE_DIR, f"{VERSION}_calibrated_temperature.pt"))
    print(f"✅ Optimal temperature: {temperature_value:.4f}")
    print(f"✅ Calibrated Log Loss: {logloss:.4f}")

    return temperature_value

# Reliability diagram
def plot_reliability_diagram(logits, labels, temperature, n_bins=15):
    probs = F.softmax(logits / temperature, dim=1)
    confidences, predictions = torch.max(probs, 1)
    accuracies = predictions.eq(labels)

    bins = torch.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bins[:-1], bins[1:]

    bin_accuracies = []
    bin_confidences = []

    for lower, upper in zip(bin_lowers, bin_uppers):
        mask = (confidences > lower) & (confidences <= upper)
        if mask.any():
            bin_accuracies.append(accuracies[mask].float().mean())
            bin_confidences.append(confidences[mask].mean())

    plt.figure(figsize=(6, 6))
    plt.plot(bin_confidences, bin_accuracies, marker='o', label='Model')
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
    plt.title("Reliability Diagram (After Temperature Scaling)")
    plt.xlabel("Confidence")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"{VERSION}_reliability_diagram_calibrated.png"))
    plt.close()
    print(f"📊 Saved reliability diagram to {VERSION}_reliability_diagram_calibrated.png")

# 🔥 Run hybrid calibration only if logits exist
if os.path.exists(f"{VERSION}_logits_eval.npy") and os.path.exists(f"{VERSION}_labels_eval.npy"):
    temp_value = apply_temperature_scaling()
    if temp_value is not None:
        logits = torch.tensor(np.load(f"{VERSION}_logits_eval.npy"), dtype=torch.float32)
        labels = torch.tensor(np.load(f"{VERSION}_labels_eval.npy"), dtype=torch.long)
        plot_reliability_diagram(logits, labels, torch.tensor(temp_value))
        # To use later: wrap model if needed
        # temp_model = ModelWithTemperature(model)
        # temp_model.temperature.data = torch.tensor([temp_value])
else:
    print("⚠️ Skipping temperature scaling and diagram (missing logits or labels)")

⚠️ Skipping temperature scaling and diagram (missing logits or labels)
